# Create Parquet references for the COAWST forecast archive on AWS Open Data
We use [kerchunk](https://fsspec.github.io/kerchunk/) to first create individual JSON reference files for each weekly NetCDF file, 
then create Parquet references that allow access as a single dataset in Xarray

In [ ]:
import fsspec
import xarray as xr

from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
from fsspec.implementations.reference import LazyReferenceMapper

from dask.distributed import Client
import dask.bag as db
from pathlib import Path
import numpy as np
import ujson

In [ ]:
import kerchunk
kerchunk.__version__

We can read from AWS Open Data using `anon=True`:

In [ ]:
fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True, use_listings_cache=False )

We can't *write* to AWS Open Data without credentials, which we will specify through environment variables.  Because we are going to use environment variables instead of referencing an AWS profile, we don't specify `profile=` here in fs_write, but use `anon=False`:

In [ ]:
fs_write = fsspec.filesystem('s3', anon=False, skip_instance_cache=True, use_listings_cache=False)

In [ ]:
flist = fs_read.glob('s3://usgs-coawst/useast-archive/*.nc')
json_dir = 's3://usgs-coawst/useast-archive/individual_jsons'

In [ ]:
flist = [f's3://{f}' for f in flist]

In [ ]:
print(len(flist))
print(flist[0])
print(flist[-1])

#### Create references for each NetCDF file in parallel 

In [ ]:
so = dict(mode='rb', anon=True, skip_instance_cache=True)

In [ ]:
def gen_json(u):
    with fs_read.open(u, **so) as infile:
        fname = Path(u).stem
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)
        outf = f'{json_dir}/{fname}.json'
        with fs_write.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());
    return outf

Import Nebari helper functions and set credentials

In [ ]:
import sys, os
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import nebari_tools as nbt

aws_profile = 'coawst_open_data'
aws_region = 'us-west-2'
endpoint_url = f's3.{aws_region}.amazonaws.com'
nbt.set_credentials(profile=aws_profile, region=aws_region, endpoint_url=endpoint_url)

Create a Dask Gateway cluster on Nebari

In [ ]:
worker_max = 30
client,cluster = nbt.start_dask_cluster(profile=aws_profile, worker_max=worker_max, 
                                      region=aws_region, use_existing_cluster=False,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      worker_profile='Small Worker', 
                                      propagate_env=True) 

Create the references for each file in parallel using Dask Bag.  Dask Bag splits a large list of tasks into partitions, and here we specify splitting the list into four partitions for each Dask worker.  

In [ ]:
%%time
bag = db.from_sequence(flist, npartitions=worker_max*4).map(gen_json)
bag.compute(retries=20)

#### Create combined references and store in Parquet files

In [ ]:
json_list = fs_read.glob(f'{json_dir}/*.json')
json_list = [f's3://{j}' for j in json_list]
print(len(json_list))
print(json_list[0])
print(json_list[-1])

In [ ]:
# fs_write.rm(json_list)  # use this if you need to start over

Create a list of variables not to combine along the time dimension

In [ ]:
ds = xr.open_dataset(fs_read.open(flist[0]))
identical_dims = []
for v in ds.variables.keys():
    if 'ocean_time' not in ds[v].dims:
        identical_dims.append(v)

Define pre and post-processing functions for the combined dataset references

In [ ]:
import zarr

def postprocess(refs):
    tmp = zarr.open(refs)
    tmp['ocean_time'].attrs['standard_name'] = 'time' 
    return refs

Create a Lazy Reference Mapper object to be written locally with a large record_size so we don't get the references for each variable split into a bunch of parts

In [ ]:
combined_parquet = 'combined.parq'

In [ ]:
out = LazyReferenceMapper.create(combined_parquet, fs=None, record_size=100000)

In [ ]:
%%time
_ = MultiZarrToZarr(
        json_list,
        remote_protocol="s3",
        concat_dims=["ocean_time"],
        coo_map={"ocean_time": "cf:ocean_time"},
        identical_dims=identical_dims,
        preprocess=kerchunk.combine.drop("dstart"),
        postprocess=postprocess,
        out=out).translate()
out.flush()

In [ ]:
combined_parquet_aws = 's3://usgs-coawst/useast-archive/combined.parq'

In [ ]:
fs_write.rm(combined_parquet_aws, recursive=True)

In [ ]:
_ = fs_write.upload(combined_parquet, combined_parquet_aws, recursive=True)

#### Test opening combined dataset:

In [ ]:
ds = xr.open_dataset(combined_parquet, engine='kerchunk', chunks={},
                    backend_kwargs=dict(storage_options=dict(target_options=so,
                    remote_protocol='s3', lazy=True, remote_options=so)))

In [ ]:
ds.ocean_time

In [ ]:
%%time
fs_ref = fsspec.implementations.reference.ReferenceFileSystem(
            combined_parquet_aws, remote_protocol="s3", target_protocol="s3", lazy=True)

ds = xr.open_dataset(fs_ref.get_mapper(), engine="zarr", backend_kwargs={"consolidated": False}, chunks={}, drop_variables=['dstart'])

ds

In [ ]:
ds.Vtransform.data

In [ ]:
import fsspec
fs = fsspec.filesystem('s3',anon=True)
xr.open_dataset(fs.open(flist[0])).Vtransform.data

#### Write Intake Catalog

In [ ]:
cat_file = "coawst_intake.yml"

dataset = 'COAWST-USEAST' 
    
ymlentry = f"""# THIS FILE AUTO-GENERATED
# This 'header' information needed to make this yml valid.  
description: 'intake catalog example'
metadata:
  version: 1
  description: "Intake Catalog for data from USGS-COAWST model"

sources:
# If you are copy/paste-ing this entry to another catalog, you only need the 
# lines from here down in your 'sources' section (be sure to indent as appropriate).
  {dataset}:
    driver: intake_xarray.xzarr.ZarrSource
    description: 'USGS COAWST US East Coast and Gulf Coast Forecast Archive'
    args:
      urlpath: ['reference://']
      consolidated: false
      drop_variables: dstart
      storage_options:
        target_options:
          anon: true
          skip_instance_cache: true
        fo: {combined_parquet_aws}
        lazy: 'true'
        remote_options:
          anon: true
          skip_instance_cache: true
        remote_protocol: 's3'
"""

with open(cat_file, 'w') as ymlfile:
   ymlfile.write(ymlentry)

In [ ]:
cat_file_s3 = f's3://usgs-coawst/useast-archive/{cat_file}'

In [ ]:
_ = fs_write.upload(cat_file, cat_file_s3)

In [ ]:
cat_file_s3

In [ ]:
cat_file_https = 'https://usgs-coawst.s3.amazonaws.com/useast-archive/coawst_intake.yml'

#### Test Intake dataset

In [ ]:
import intake

In [ ]:
cat = intake.open_catalog(cat_file_https)

In [ ]:
list(cat)

In [ ]:
ds = cat['COAWST-USEAST'].to_dask()

In [ ]:
ds

#### Shut down cluster

In [ ]:
client.close();   cluster.shutdown()